In [ ]:
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

In [ ]:
QPATH = "Quantlet/4-qode2desc"

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../src')

In [ ]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

from sklearn.model_selection import train_test_split


In [ ]:
import analysis_modules
importlib.reload(analysis_modules)

In [ ]:
model_name = "CodeT5"
SAMPLE = 'test'


# tokenization
encoder_max_length = 500
decoder_max_length = 100
RS = 42
LR = 5e-5

EPOCHS = 7
TRAIN_BATCH = 4
EVAL_BATCH  = 4

WARMUP_STEPS  = 500
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True

MODE = 'no_context'

EVAL_COLUMNS = ['eval_loss',
                'eval_rouge1',
                'eval_rouge2',
                'eval_rougeL',
                'eval_rougeLsum',
                'eval_bleu',
                'eval_gen_len']

DATE = str(datetime.date.today())
analysis_name = model_name + '-' + MODE + '-' + SAMPLE  + '-' + str(EPOCHS) + '-' + str(encoder_max_length)

analysis_name = analysis_name + '-' + DATE + '-' + 'v0'
print(analysis_name)

In [ ]:
print(analysis_name)
analysis_modules.scs_analyze(analysis_name=analysis_name,
                                    model_name=model_name,
                                    train_data_path=f'../../data/preprocessed/Quantlet/{MODE}/',
                                    train_data_name='full_train_dataset_20230923.json',
                                    val_data_path=f'../../data/preprocessed/Quantlet/{MODE}/',
                                    val_data_name='test_dataset_20230923.json',
                                    encoder_max_length=encoder_max_length,
                                    decoder_max_length=decoder_max_length,
                                    random_state=RS,
                                    eval_columns_list=EVAL_COLUMNS,
                                    learning_rate=LR,
                                    epochs=EPOCHS,
                                    train_batch=TRAIN_BATCH,
                                    eval_batch=EVAL_BATCH,
                                    warmup_steps=WARMUP_STEPS,
                                    weight_decay=WEIGHT_DECAY,
                                    logging_stes=LOGGING_STEPS,
                                    save_total_lim=SAVE_TOTAL_LIM,
                                    label_smooting=LABEL_SMOOTHING,
                                    predict_generate=PREDICT_GENERATE)